In [20]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cft
import IPython.display

In [22]:
session = cc.database.create_session()

In [43]:
from dask_jobqueue import PBSCluster

walltime = '01:00:00'
cores = 4
memory = '16GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),'-l mem='+str(memory),
                                '-P v45',
                                '-l storage=gdata/v45+scratch/v45+gdata/hh5+gdata/ik11'],
                     header_skip=["select"])

In [45]:
cluster

In [68]:
from distributed import Client

client=Client(cluster)
client

Client Scheduler: tcp://10.6.1.51:35499 Dashboard: http://10.6.1.51:8787/status,Cluster Workers: 4 Cores: 16 Memory: 64.00 GB


In [69]:
temp = cc.querying.getvar('01deg_jra55v140_iaf', 'temp', session,
                    start_time='1996-01-01 00:00:00', end_time='2000-01-01 00:00:00', frequency='1 monthly'
                   )
temp

  warnings.warn(



,Array,Chunk
Bytes,157.46 GB,1.85 MB
Shape,"(54, 75, 2700, 3600)","(1, 19, 135, 180)"
Count,172818 Tasks,86400 Chunks
Type,float32,numpy.ndarray


In [70]:
temp_mean = temp.mean('time')

In [71]:
temp_mean

,Array,Chunk
Bytes,2.92 GB,1.85 MB
Shape,"(75, 2700, 3600)","(19, 135, 180)"
Count,289618 Tasks,1600 Chunks
Type,float32,numpy.ndarray


In [72]:
temp_anom = temp - temp_mean

In [ ]:
temp_anom.isel(st_ocean=0).plot(bins=50)

distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/g/data/v45/conda/envs/cosima/lib/python3.8/site-packages/distributed/utils.py", line 656, in log_errors
    yield
  File "/g/data/v45/conda/envs/cosima/lib/python3.8/site-packages/distributed/scheduler.py", line 3511, in retire_workers
    await self.replicate(
  File "/g/data/v45/conda/envs/cosima/lib/python3.8/site-packages/distributed/scheduler.py", line 3270, in replicate
    assert count > 0
AssertionError
distributed.core - ERROR - 
Traceback (most recent call last):
  File "/g/data/v45/conda/envs/cosima/lib/python3.8/site-packages/distributed/core.py", line 513, in handle_comm
    result = await result
  File "/g/data/v45/conda/envs/cosima/lib/python3.8/site-packages/distributed/scheduler.py", line 3511, in retire_workers
    await self.replicate(
  File "/g/data/v45/conda/envs/cosima/lib/python3.8/site-packages/distributed/scheduler.py", line 3270, in replicate
    assert count > 0
AssertionError
distribut

In [67]:
client.restart()

Client Scheduler: tcp://10.6.1.51:35499 Dashboard: http://10.6.1.51:8787/status,Cluster Workers: 4 Cores: 16 Memory: 64.00 GB
